In [1]:
import pandas as pd
import os
import geohash
import numpy as np

import warnings

warnings.filterwarnings('ignore')
processedDf = df = pd.read_csv('/home/data/train_new.csv',
                 usecols=['start_time', 'end_time', 'start_lat', 'start_lon', 'end_lat', 'end_lon'])
processedDf.drop('end_time',axis=1,inplace=True)
processedDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495814 entries, 0 to 1495813
Data columns (total 5 columns):
start_time    1495814 non-null object
start_lat     1495814 non-null float64
start_lon     1495814 non-null float64
end_lat       1495814 non-null float64
end_lon       1495814 non-null float64
dtypes: float64(4), object(1)
memory usage: 57.1+ MB


In [2]:
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361


In [3]:
dateDf = pd.DataFrame()
dateDf['date']=pd.to_datetime(processedDf['start_time'])

In [4]:
dateDf.head()

,date
0,2018-01-20 10:13:43
1,2018-02-12 17:40:51
2,2018-02-13 14:52:24
3,2018-02-13 17:23:08
4,2018-02-13 18:06:02


In [5]:
dateDf['date']=pd.to_datetime(dateDf['date']).dt.weekday

In [6]:
dateDf.head()

,date
0,5
1,0
2,1
3,1
4,1


In [7]:
weekendDf = pd.DataFrame()
weekend={0:0,
         1:0,
         2:0,
         3:0,
         4:0,
         5:1,
         6:1
         
}
weekendDf['weekend'] = dateDf['date'].map(weekend)

In [8]:
weekendDf.head()

,weekend
0,1
1,0
2,0
3,0
4,0


In [9]:
processedDf = pd.concat([processedDf,weekendDf],axis=1)
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0


In [11]:
yearDf = pd.DataFrame()
monthDf = pd.DataFrame()
dayDf = pd.DataFrame()
strDf = pd.DataFrame()
dateDf['date']=pd.to_datetime(processedDf['start_time'])#重新初始化
yearDf['year']=pd.to_datetime(dateDf['date']).dt.year
monthDf['month']=pd.to_datetime(dateDf['date']).dt.month
dayDf['day']=pd.to_datetime(dateDf['date']).dt.day
strDf['day']=pd.to_datetime(dateDf['date']).dt.day

In [12]:
processedDf = pd.concat([processedDf,yearDf],axis=1)
processedDf = pd.concat([processedDf,monthDf],axis=1)
processedDf = pd.concat([processedDf,dayDf],axis=1)
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13


In [13]:
hourDf = pd.DataFrame()
minuteDf = pd.DataFrame()
hourDf['hour']=pd.to_datetime(dateDf['date']).dt.hour
minuteDf['minute']=pd.to_datetime(dateDf['date']).dt.minute

In [14]:
hourDf.head()

,hour
0,10
1,17
2,14
3,17
4,18


以下是time_cat测试代码

In [15]:
hourDf = pd.concat([hourDf,minuteDf],axis=1)
hourDf.head()

,hour,minute
0,10,13
1,17,40
2,14,52
3,17,23
4,18,6


In [16]:
hourDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495814 entries, 0 to 1495813
Data columns (total 2 columns):
hour      1495814 non-null int64
minute    1495814 non-null int64
dtypes: int64(2)
memory usage: 22.8 MB


In [17]:
def time_cat(hour,minute):
    hour = str(hour)
    minute = str(int(minute/15)*15)
    return hour + ":" + minute


In [18]:
hourDf['time_cat'] = hourDf.apply(lambda row:time_cat(row['hour'],row['minute']),axis=1)
hourDf.head()

,hour,minute,time_cat
0,10,13,10:0
1,17,40,17:30
2,14,52,14:45
3,17,23,17:15
4,18,6,18:0


In [19]:
processedDf.loc[:,"time_cat"] = hourDf.loc[:,"time_cat"]
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0


In [20]:
def time_num(hour,minute):
    per = (hour*60+minute)/(60*24)
    return per

In [21]:
hourDf['time_num']=hourDf.apply(lambda row:time_num(row['hour'],row['minute']),axis=1)
processedDf.loc[:,"time_num"] = hourDf.loc[:,"time_num"]

In [22]:
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat,time_num
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0,0.425694
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30,0.736111
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45,0.619444
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15,0.724306
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0,0.754167


In [23]:
import math
def time_cos(time_num):
    item = math.cos(time_num*2*math.pi)
    return item
def time_sin(time_num):
    item = math.sin(time_num*2*math.pi)
    return item

In [24]:
hourDf['time_cos']=hourDf.apply(lambda row:time_cos(row['time_num']),axis=1)
hourDf['time_sin']=hourDf.apply(lambda row:time_sin(row['time_num']),axis=1)
processedDf.loc[:,"time_cos"] = hourDf.loc[:,"time_cos"]
processedDf.loc[:,"time_sin"] = hourDf.loc[:,"time_sin"]
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat,time_num,time_cos,time_sin
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0,0.425694,-0.892979,0.450098
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30,0.736111,-0.087156,-0.996195
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45,0.619444,-0.731354,-0.681998
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15,0.724306,-0.160743,-0.986996
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0,0.754167,0.026177,-0.999657


In [25]:
dateDf.head()

,date
0,2018-01-20 10:13:43
1,2018-02-12 17:40:51
2,2018-02-13 14:52:24
3,2018-02-13 17:23:08
4,2018-02-13 18:06:02


In [26]:
dateDf['day']=pd.to_datetime(dateDf['date']).dt.weekday
day_to_str = {0: "Monday",
              1: "Tuesday",
              2: "Wednesday",
              3: "Thursday",
              4: "Friday",
              5: "Saturday",
              6: "Sunday"}
dateDf['day'] = dateDf['day'].map(day_to_str)
dateDf.head()

,date,day
0,2018-01-20 10:13:43,Saturday
1,2018-02-12 17:40:51,Monday
2,2018-02-13 14:52:24,Tuesday
3,2018-02-13 17:23:08,Tuesday
4,2018-02-13 18:06:02,Tuesday


In [27]:
processedDf.loc[:,"day_cat"] = dateDf.loc[:,"day"]

In [28]:
hourDf['day_index']=pd.to_datetime(dateDf['date']).dt.weekday
hourDf.head()

,hour,minute,time_cat,time_num,time_cos,time_sin,day_index
0,10,13,10:0,0.425694,-0.892979,0.450098,5
1,17,40,17:30,0.736111,-0.087156,-0.996195,0
2,14,52,14:45,0.619444,-0.731354,-0.681998,1
3,17,23,17:15,0.724306,-0.160743,-0.986996,1
4,18,6,18:0,0.754167,0.026177,-0.999657,1


In [29]:
def day_num(time_num,day_index):
    per = (day_index+time_num)/7
    return per

In [30]:
hourDf['day_num']=hourDf.apply(lambda row:day_num(row['time_num'],row['day_index']),axis=1)
processedDf.loc[:,"day_num"] = hourDf.loc[:,"day_num"]
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595


In [31]:
def day_cos(day_num):
    per = math.cos(day_num*2*math.pi)
    return per
def day_sin(day_num):
    per = math.sin(day_num*2*math.pi)
    return per

In [32]:
hourDf['day_cos']=hourDf.apply(lambda row:day_cos(row['day_num']),axis=1)
hourDf['day_sin']=hourDf.apply(lambda row:day_sin(row['day_num']),axis=1)
processedDf.loc[:,"day_cos"] = hourDf.loc[:,"day_cos"]
processedDf.loc[:,"day_sin"] = hourDf.loc[:,"day_sin"]
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099,0.157050,-0.987591
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159,0.789543,0.613695
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349,0.116918,0.993142
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329,0.023061,0.999734
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595,-0.003740,0.999993


In [33]:
def x_start(lat_start,lon_start):
    per = math.cos(lat_start)*math.cos(lon_start)
    return per
def y_start(lat_start,lon_start):
    per = math.cos(lat_start)*math.sin(lon_start)
    return per
def z_start(lat_start):
    per = math.sin(lat_start)
    return per

In [34]:
processedDf['x_start']=processedDf.apply(lambda row:x_start(row['start_lat'],row['start_lon']),axis=1)
processedDf['y_start']=processedDf.apply(lambda row:y_start(row['start_lat'],row['start_lon']),axis=1)
processedDf['z_start']=processedDf.apply(lambda row:z_start(row['start_lat']),axis=1)
processedDf.head()

,start_time,start_lat,start_lon,end_lat,end_lon,weekend,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,x_start,y_start,z_start
0,2018-01-20 10:13:43,33.783415,111.603660,33.779811,111.605885,1,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099,0.157050,-0.987591,-0.055090,0.712922,0.699076
1,2018-02-12 17:40:51,34.810763,115.549264,34.814875,115.549374,0,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159,0.789543,0.613695,0.746854,-0.615984,-0.250546
2,2018-02-13 14:52:24,34.640284,115.539024,34.813136,115.559243,0,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349,0.116918,0.993142,0.762286,-0.641939,-0.082670
3,2018-02-13 17:23:08,34.818280,115.542039,34.813141,115.559217,0,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329,0.023061,0.999734,0.740919,-0.620137,-0.257816
4,2018-02-13 18:06:02,34.813278,115.559260,34.786126,115.874361,0,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595,-0.003740,0.999993,0.752481,-0.608090,-0.252980


In [35]:
def location(lat,lon):
    per = geohash.encode(lat,lon,9)
    return per

In [36]:
processedDf['location_start']=processedDf.apply(lambda row:location(row['start_lat'],row['start_lon']),axis=1)
processedDf['location_end']=processedDf.apply(lambda row:location(row['end_lat'],row['end_lon']),axis=1)

In [37]:
processedDf.drop(['start_time','start_lat','start_lon','end_lat','end_lon'],axis=1,inplace=True)
processedDf.head()

,weekend,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,x_start,y_start,z_start,location_start,location_end
0,1,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099,0.157050,-0.987591,-0.055090,0.712922,0.699076,wqp25w569,wqp25tkvt
1,0,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159,0.789543,0.613695,0.746854,-0.615984,-0.250546,ww4nj3h7m,ww4nj3u7r
2,0,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349,0.116918,0.993142,0.762286,-0.641939,-0.082670,ww4jj4hfq,ww4nj9e9j
3,0,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329,0.023061,0.999734,0.740919,-0.620137,-0.257816,ww4nj4rph,ww4nj9e9h
4,0,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595,-0.003740,0.999993,0.752481,-0.608090,-0.252980,ww4nj9edj,ww4muu95u


In [38]:
processedDf=processedDf.reindex_axis(["year","month","day",
                                      "time_cat","time_num","time_cos","time_sin",
                                      "day_cat","day_num","day_cos","day_sin",
                                      "weekend",
                                      "x_start","y_start","z_start",
                                      "location_start","location_end"],axis=1)
processedDf.head()

,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,x_start,y_start,z_start,location_start,location_end
0,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099,0.157050,-0.987591,1,-0.055090,0.712922,0.699076,wqp25w569,wqp25tkvt
1,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159,0.789543,0.613695,0,0.746854,-0.615984,-0.250546,ww4nj3h7m,ww4nj3u7r
2,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349,0.116918,0.993142,0,0.762286,-0.641939,-0.082670,ww4jj4hfq,ww4nj9e9j
3,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329,0.023061,0.999734,0,0.740919,-0.620137,-0.257816,ww4nj4rph,ww4nj9e9h
4,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595,-0.003740,0.999993,0,0.752481,-0.608090,-0.252980,ww4nj9edj,ww4muu95u


In [39]:
processedDf.to_csv('featured-dataset_new.csv')